# **Introduction**
-----------------------------------------------------
The following protocol describes computational methods used for analysis of per residues energy of VHH-antigen complexes for used as a references according to a proposal " rational design of human-made focused library towards specific types of antibodies". The protocol will capture Rosetta protocols applied for VHH analysis. All Rosetta codes used in this publication refers to version 3.8. All materials shown here can be downloaded from *https://github.com/Karnpob-coder/ROSETTA*

# **Structure download and preparation**
-------------------------------------------------------
All VHH structures in this study were retrieved from the structural antibody Database (SabDab) using searching filters as (1) VHH antibodies; (2) sub-3.5 A resolution, (3) antigen type protein or peptide, (4) non-redundant exclusion. Clustering of variable domain region, based on these amino acid sequences, was conducted using CD-HIT *http://weizhong-lab.ucsd.edu/cd-hit/* to eliminate redundancies by removing perfect matches for which the % id was 100%. PDB files of these filtered structures were download from the Protein DataBank followed by pre-processing that includes removing water and non-protein residues and residue re-numbering to start from residue 1. Then, structures were relaxed to avoid any clashed residues using a constrained FastRelax refinement protocol which restraint to starting coordinates of backbone VH atoms at a standard deviation of 0.5A. Finally, 10 relaxed structures were built and used an input complex. The best scoring relaxed structure was used for binding energy per residue analysis. The XML script and codes described below were used to run relaxation. Note that Binding energy per residue was calculated using a custom RosettaScripts XML protocol (Fleishman et al., 2011) that utilizes the REF2015 score function (Alford et al., 2017).

rlax.command:

In [ ]:
rosetta_scripts.default.linuxgccrelease @rlax.options \ 
-s 4ZG1_HA.pdb -parser:protocol rlax_cst.xml

rlax.options:

In [ ]:
-in:file:fullatom
-out:file:fullatom
-out:pdb_gz
-ex1
-use_input_sc
-nstruct 10
-suffix _rlax_iface
-scorefile rlax_iface.fasc
-out:path:pdb models/

rlax_cst.xml:

In [ ]:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="ref_cst" weights="ref2015_cst.wts" />
		<ScoreFunction name="ref" weights="ref2015.wts" />
	</SCOREFXNS>
	<FILTERS>
	</FILTERS>
	<TASKOPERATIONS>
		<InitializeFromCommandline name="ifcl"/>
		<RestrictToRepacking name="rtr"/>
		<RestrictToInterfaceVector name="rtiv" chain1_num="1" chain2_num="2" CB_dist_cutoff="10.0" nearby_atom_cutoff="5.5" vector_angle_cutoff="75" vector_dist_cutoff="9.0" />
	</TASKOPERATIONS>
	<MOVERS>
		<PackRotamersMover name="repack" task_operations="ifcl,rtr" scorefxn="ref" />
		<FastRelax name="relax" task_operations="ifcl,rtr,rtiv" scorefxn="ref_cst" >
			<MoveMap name="mm" >
				<Jump number="1" setting="0" />
			</MoveMap>
		</FastRelax>
		<AtomCoordinateCstMover name="cst" />
		<VirtualRoot name="root" removable="1" />
		<VirtualRoot name="rmroot" remove="1" />
	</MOVERS>
	<PROTOCOLS>
		<Add mover_name="repack" />
		<Add mover_name="root" />
		<Add mover_name="cst" />
		<Add mover_name="relax"/>
		<Add mover_name="rmroot" />
	</PROTOCOLS>
	<OUTPUT scorefxn="ref" />
</ROSETTASCRIPTS>

# **Binding energy measurement**
-----------------------------------
To determine binding energy at the VHH-antigen interface, below XML script and codes shown below were used to measure the binding energy over the binding interface and per-residues energy of each interface residue.

en_measure.command:

In [ ]:
rosetta_scripts.mpi.linuxgccrelease @en_measure.options \
-l best_decoys.list -scorefile ddg.fasc  \
-parser:protocol en_measure.xml

en_measure.options:

In [ ]:
-in:file:fullatom
-out:file:fullatom
-out:pdb_gz
-ex1
-use_input_sc
-nstruct 1
-out:file:score_only
-out:no_nstruct_label

en_measure.xml:

In [ ]:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="ref_cst" weights="ref2015_cst.wts" />
		<ScoreFunction name="ref" weights="ref2015.wts" />
	</SCOREFXNS>
	<FILTERS>
	</FILTERS>
	<TASKOPERATIONS>
		<InitializeFromCommandline name="ifcl"/>
		<RestrictToRepacking name="rtr"/>
		<RestrictToInterfaceVector name="rtiv" chain1_num="1" chain2_num="2" CB_dist_cutoff="10.0" nearby_atom_cutoff="5.5" vector_angle_cutoff="75" vector_dist_cutoff="9.0" />
	</TASKOPERATIONS>
	<MOVERS>
		<ddG name="ddg" per_residue_ddg="1" repack_unbound="1" chain_num="1" task_operations="rtiv,ifcl,rtr" scorefxn="ref" />
	</MOVERS>
	<PROTOCOLS>
		<Add mover_name="ddg" />
	</PROTOCOLS>
	<OUTPUT scorefxn="ref" />
</ROSETTASCRIPTS>